In [ ]:
import numpy as np
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"  # specify which GPU(s) to be used


In [ ]:
## Preprocess
from pyvi import ViTokenizer, ViPosTagger

def preprocess(text):
    text = text.lower()
    text = ViTokenizer.tokenize(text)
    return text

In [27]:
f = open('ViQuAD/train_ViQuAD.json', 'r')
train_set = json.load(f)
train_set = train_set['data']
f = open('ViQuAD/dev_ViQuAD.json', 'r')
dev_set = json.load(f)
dev_set = dev_set['data']
f = open('ViQuAD/test_ViQuAD.json', 'r')
test_set = json.load(f)
test_set = test_set['data']

In [35]:
# contexts = []
id = 0
all_data = train_set+dev_set+test_set
for article in all_data:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        # contexts.append(
        #     {'id': id,
        #     'contents': context,
        #     'title': title
        # })
        contexts[id]['ori_contents'] = preprocess(context)
        id+=1
len(contexts)

5109

In [29]:
from tqdm.auto import tqdm

for context in tqdm(contexts):
    content = context['contents']

    pred_text = create_queries(content)
    pred_text = ' '.join(pred_text)
    context['contents'] = preprocess(content + ' ' + pred_text)

In [36]:
contexts[0]

{'id': 0,
 'contents': 'phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm 1976 ( từ năm 1981 gọi là chủ_tịch hội_đồng_bộ_trưởng ) cho đến khi nghỉ hưu năm 1987 . trước đó ông từng giữ chức_vụ thủ_tướng chính_phủ việt nam dân_chủ cộng_hòa từ năm 1955 đến năm 1976 . ông là vị thủ_tướng việt_nam tại vị lâu nhất ( 1955 – 1987 ) . ông là học_trò , cộng_sự của chủ_tịch hồ chí minh . ông có tên gọi thân_mật là tô , đây từng là bí_danh của ông . ông còn có tên gọi là lâm bá kiệt khi làm phó chủ_nhiệm cơ_quan biện sự xứ tại quế lâm ( chủ_nhiệm là hồ học lãm ) . tên của thủ_tướng việt nam là gì ai là chủ_tịch chính_phủ đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt nam thủ_tướng việt nam là ai thủ_tướng việt nam là ai phạm văn đồng là ai',
 'title': 'Phạm Văn Đồng',
 'ori_contents': 'phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm

In [37]:
with open('collections/doc2query.json','w') as f:
    json.dump(contexts, f)


In [26]:
contexts[0]

{'id': 0,
 'contents': 'phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm 1976 ( từ năm 1981 gọi là chủ_tịch hội_đồng_bộ_trưởng ) cho đến khi nghỉ hưu năm 1987 . trước đó ông từng giữ chức_vụ thủ_tướng chính_phủ việt nam dân_chủ cộng_hòa từ năm 1955 đến năm 1976 . ông là vị thủ_tướng việt_nam tại vị lâu nhất ( 1955 – 1987 ) . ông là học_trò , cộng_sự của chủ_tịch hồ chí minh . ông có tên gọi thân_mật là tô , đây từng là bí_danh của ông . ông còn có tên gọi là lâm bá kiệt khi làm phó_chủ_nhiệm cơ_quan biện sự xứ tại quế lâm ( chủ_nhiệm là hồ học lãm ) . python là gì python là gì ? ai đã tạo_ra ngôn_ngữ python ? python là gì ngôn_ngữ nào là python python được thiết_kế để làm gì python là gì python là gì python là gì python là gì ?',
 'title': 'Phạm Văn Đồng'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = 'doc2query/msmarco-vietnamese-mt5-base-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')


In [ ]:
import time
start = time.time()
text = "Python (phát âm tiếng Anh: /ˈpaɪθɑːn/) là một ngôn ngữ lập trình bậc cao cho các mục đích lập trình đa năng, do Guido van Rossum tạo ra và lần đầu ra mắt vào năm 1991. Python được thiết kế với ưu điểm mạnh là dễ đọc, dễ học và dễ nhớ. Python là ngôn ngữ có hình thức rất sáng sủa, cấu trúc rõ ràng, thuận tiện cho người mới học lập trình và là ngôn ngữ lập trình dễ học; được dùng rộng rãi trong phát triển trí tuệ nhân tạo. Cấu trúc của Python còn cho phép người sử dụng viết mã lệnh với số lần gõ phím tối thiểu. Vào tháng 7 năm 2018, van Rossum đã từ chức lãnh đạo trong cộng đồng ngôn ngữ Python sau 30 năm làm việc."


def create_queries(para, type_gen='sampling', ):
    input_ids = tokenizer.encode(para, return_tensors='pt').to('cuda:0')
    outputs = []
    with torch.no_grad():
        if type_gen == 'sampling':
            # Here we use top_k / top_k random sampling. It generates more diverse queries, but of lower quality
            pred_outputs = model.generate(
                input_ids=input_ids,
                max_length=64,
                do_sample=True,
                top_p=0.95,
                top_k=10, 
                num_return_sequences=5
                ).to('cuda')
        else:
            # Here we use Beam-search. It generates better quality queries, but with less diversity
            pred_outputs = model.generate(
                input_ids=input_ids, 
                max_length=64, 
                num_beams=5, 
                no_repeat_ngram_size=2, 
                num_return_sequences=5, 
                early_stopping=True
            )

        for i in range(len(pred_outputs)):
            query = tokenizer.decode(pred_outputs[i], skip_special_tokens=True)
            outputs.append(query)

    return outputs

create_queries(text)
print(time.time()-start)

In [ ]:
## Index
# !python -m pyserini.index.lucene \
#     --collection JsonCollection \
#     --input collections \
#     --index indexes/sample_collection_jsonl \
#     --generator DefaultLuceneDocumentGenerator \
#     --threads 1 \
#     --storePositions --storeDocvectors --storeRaw